# Библиотеки

In [13]:
from zlib import crc32

import numpy as np
import pandas as pd
import scipy.stats as st
from scipy.stats import jarque_bera
from scipy.stats import shapiro
from sklearn import preprocessing

# Код генерации выборки

In [2]:
import re
EMAIL_REGEX = re.compile(r"[^@]+@phystech.edu")

def generate_dataset(code):
    rs = np.random.RandomState(code)
    data = rs.randn(100)
    noise_case = rs.randint(low = 0, high = 3)
    max_attemp_num = 10000
    if noise_case == 0:
        bad = True
        attemps = 0
        while bad and attemps < max_attemp_num:
            data2 = data + rs.laplace(size = (100)) * 0.25
            p1 = jarque_bera(data2)[1]
            p2 = shapiro(data2)[1]
            
            if p1 < 0.05 and p2 > 0.05:
                bad = False
        data = data2
        if bad:
            print ('could not')
    elif noise_case == 1:
        data = data + rs.gamma(1, size=(100))*0.5
    elif noise_case == 2:
        data = data +  rs.uniform()*1.5
        
    return data

# Генерация выборки для вашей почты

<span style="color:red">
    ВАЖНО!
    Почта, которую укажете ниже и почта с которой Вы отправляете домашнее задание должна совпадать!
    В момент проверки задания алгоритм генерирует выборку на основе почты из анкеты!
</span>

Внимательно проверьте почту для которой выполняется задание!

In [3]:
task = dict()
task['mail'] = input(prompt='Enter your mail: ')
assert EMAIL_REGEX.match(task['mail']), 'Not a mail in the phystech.edu domain'
task['id'] = crc32(task['mail'].encode('utf-8'))
task['data'] = generate_dataset(task['id'])

task

Enter your mail: bogdanov.ai@phystech.edu


{'mail': 'bogdanov.ai@phystech.edu',
 'id': 1162440694,
 'data': array([-9.61774953e-01,  6.69269582e-01,  7.52536655e-01,  7.72096892e-01,
         2.48307824e-01,  4.27275676e-01, -3.68268132e-01,  2.68664185e-01,
         7.63657750e-01, -2.71434053e-01, -1.02881203e+00,  1.11595523e+00,
        -7.36590113e-01,  1.56319505e+00,  1.89837670e+00,  7.23331331e-01,
        -7.79337322e-01,  2.44787707e+00, -4.64604442e-01, -9.74178889e-01,
        -7.33371037e-01,  7.73970306e-01,  3.18752138e-01,  7.51634790e-01,
        -1.99153929e-01,  1.29853253e+00,  6.31625806e-02,  1.21256654e+00,
         1.22007858e+00,  1.25859743e+00,  1.21085906e+00,  2.48274891e+00,
         2.38067259e+00,  4.02601428e-01, -3.33556024e-01,  9.72038927e-01,
         2.77289621e+00,  7.08850403e-01, -1.47917564e+00,  2.14859409e+00,
         8.00274537e-01,  1.12240139e+00, -1.60297937e+00,  1.86434042e+00,
         3.80100119e-01,  7.67117960e-01,  1.73131646e-01,  1.83609421e-03,
        -6.46675174e-01,

# Работа с выборкой

In [4]:
data = task['data']

## Выборочное среднее по выборке

In [5]:
print('mean={}'.format(np.mean(data)))

mean=0.5876314864271288


## Оценка стандартного отклонения (несмещенная)

In [6]:
print('std={}'.format(np.std(data, ddof=1)))

std=1.0647148883376776


## Можно ли отклонить гипотезу о нормальности выборки, используя критерий Харке-Бера (p<=0.05)?

In [43]:
jarque_bera(data).pvalue <= 0.05

False

## Можно ли отклонить гипотезу о нормальности выборки, используя критерий Шапиро-Уилка (p<=0.05)?

In [42]:
shapiro(data).pvalue <= 0.05

False

## Сумма единиц в бинаризованной выборке (threshold=0.5)

In [21]:
binarized_data = preprocessing.Binarizer(threshold=0.5).transform(data.reshape(1, -1))
summa = binarized_data.sum()
print(summa)

53.0


## Значение p-value для биномиального теста с двусторонней альтернативой (p0 = 0.5)

In [28]:
print(st.binomtest(int(summa), len(data), 0.5, alternative='two-sided').pvalue)

0.6172994135892521
